In [1]:
#General imports needed for path
import os 
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import itertools

import warnings
warnings.filterwarnings('ignore')

In [3]:
#Packages created by our group:
import route_dynamics.route_elevation.base_df as base
import route_dynamics.route_energy.knn as knn
from route_dynamics.route_riders import route_riders as ride
import route_dynamics.route_energy.longi_dynam_model as ldm
import route_dynamics.route_energy.accel as ca

In [4]:
#User defines what routes they want to evaluate
rt_list = [22,101,102,143,150,153,154,156,157,158,159,168,169,177,178,179,180,181,182,183,186,187,190,192]

#Import route shapefile

shapefile_name = '../data/rt' + str(rt_list[0]) + '_pts2.shp'

#Import bus stops shapefile

stops_shp = '../data/Transit_Stops_for_King_County_Metro__transitstop_point.shp'

signals_shp = '../data/traffic_signals2.shp'

trip183 = pd.read_csv("../data/Trip183.csv", usecols = ['SignRt', 'InOut', 'KeyTrip', 'BusType', 'Seats', 
                     'Period', 'AnnRides']) # KCM Data
trip183unsum = pd.read_csv("../data/Zon183Unsum.csv", usecols = ['Route', 'Dir', 'Trip_ID', 'InOut', 'STOP_SEQ', 'STOP_ID',
                     'Period', 'AveOn', 'AveOff', 'AveLd', 'Obs'])

#Acceleration profile
a_prof = pd.read_csv("../data/acceleration.csv", names=['time (s)', 'accel. (g)'])

In [5]:
trip_df = ride.route_ridership('PM', rt_list[0], 14000)
geometry = trip_df.geometry.values

In [6]:
xy = []
for i in range(len(geometry)):
    dic = mapping(geometry[i][0])
    coords = dic['coordinates']
    xy.append(coords)
    xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
    xy_df['coordinates'] = xy
    
xy_df

stop_coords = xy_df.coordinates.values

In [7]:
len(stop_coords)

88

In [8]:
np.shape(stop_coords)

(88,)

In [9]:
signals = gpd.read_file(signals_shp)
signals = signals[signals['Route_Num']==str(rt_list[0])]
geometry = signals.geometry.values
xy = []
for i in range(len(geometry)):
    dic = mapping(geometry[i])
    coords = dic['coordinates']
    xy.append(coords)
    xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
    xy_df['coordinates'] = xy
    
xy_df

signal_coords = xy_df.coordinates.values

In [10]:
len(signal_coords)

16

In [18]:
mass_array = trip_df.Total_Mass.values

In [19]:
# mass_array = np.array([mass_array])
# mass_array

In [20]:
# mass_array = np.swapaxes(mass_array, 0, 1)

In [21]:
np.shape(mass_array)

(88,)

In [22]:
np.shape(mass_array[0])

()

In [23]:
a_prof.head()

,time (s),accel. (g),accel. (m/s^2),vel. (m/s),vel. (mph),dist. (m)
0,0.0,0.0006,0.005886,0.000000,0.000000,0.000000
1,1.0,0.1156,1.134036,0.569961,1.253914,0.284981
2,2.0,0.1239,1.215459,1.744709,3.838359,1.442315
3,3.0,0.1261,1.237041,2.970959,6.536109,3.800149
4,4.0,0.1256,1.232136,4.205547,9.252203,7.388402


In [24]:
ex = ldm.RouteTrajectory(
rt_list[0],
shapefile_name,
a_prof,
stop_coords,
signal_coords,
mass_array
)

ValueError: shape mismatch: value array of shape (1,2) could not be broadcast to indexing result of shape (1,)